In [4]:
import rasterio
import geopandas as gpd
from rasterio.mask import mask
import os
import tqdm
import rasterio.features
from shapely.geometry import shape
import pandas as pd


os.chdir(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\0.external_data\GOSIF_GPP_SD')

#filter the raster files
raster_files = [f for f in os.listdir() if f.endswith('.tif')]

fourth_index = len(raster_files) // 4
middle_index = len(raster_files) // 2
three_fourth_index = len(raster_files) // 4 * 3

raster_files = raster_files[three_fourth_index:]

#create a empty geodataframe to store the results
results = gpd.GeoDataFrame()
shp_file = '../../1.Initilization/intermediate/unique_polygons_shapefile.shp'
gdf_mask = gpd.read_file(shp_file)
gdf_mask = gdf_mask.to_crs("EPSG:4326")
#loop through the raster files


for raster_file in tqdm.tqdm(raster_files):
    # Read the raster file (.tif)
    src = rasterio.open(raster_file)
    # Get the geometry of the shapefile as GeoJSON
    mask_geom = gdf_mask.geometry


    def raster_to_polygons(raster_array, transform, nodata=None):
        polygons = []
        for vec, val in rasterio.features.shapes(raster_array, mask=raster_array != nodata, transform=transform):
            if val != nodata:
                polygons.append({"geometry": shape(vec), "value": val})
        return polygons

    # Mask the raster using the geometry
    out_image, out_transform = mask(src, mask_geom, crop=True, filled=False)
    out_meta = src.meta.copy()
    out_meta.update({"driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform})
    polygons = raster_to_polygons(out_image[0], out_transform)
    # Create a GeoDataFrame from the polygons
    gdf = gpd.GeoDataFrame(polygons, crs=out_meta["crs"])

    # Set a new index if desired
    result = gdf.reset_index().rename(columns={0: 'value'})


    # rename the column,set name
    result.rename(columns={'index':'ID','value':'GOSIF_GPP_SD'},inplace=True)


    # Perform a spatial join between the points and polygons
    spatial_join_gdf = gpd.sjoin(result, gdf_mask, how='inner', op='within')

    # Group by the 'polygon_id' column and calculate the mean of the 'value' column
    aggregated_gdf = spatial_join_gdf.groupby('FID').agg({'GOSIF_GPP_SD': 'mean'}).reset_index()

    # join aggregated_gdf with gdf_mask on FID
    aggregated_gdf = gdf_mask.merge(aggregated_gdf, on='FID', how='left')

    # add a column to indicate the raster file
    aggregated_gdf['time'] = raster_file  
    
    #append the results to the geodataframe
    results = pd.concat([results, aggregated_gdf], ignore_index=True, axis=0)




# output the results as geojson file
results.to_file('GOSIF_GPP_SD_4.geojson', driver='GeoJSON')

  0%|          | 0/18 [00:00<?, ?it/s]c:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
  6%|▌         | 1/18 [01:04<18:20, 64.74s/it]c:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
 11%|█         | 2/18 [02:10<17:20, 65.05s/it]c:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Pleas